In [1]:
import aiohttp
import random
import sys
import torch
from PIL import Image
from matplotlib import cm
import numpy as np

sys.path.append('..')

In [2]:
from comfy_script.runtime.real import load as load_comfy
from comfy_script.client import Client

In [ ]:
comfy_url = 'http://34.84.71.113'
client = Client(comfy_url, session_factory=lambda: aiohttp.ClientSession(
    auth=aiohttp.BasicAuth('wunderbar', 'qfA=!-gyWw$8Pk[')
))

In [4]:
load_comfy(client)

In [5]:
from comfy_script.runtime.real.nodes import (
    CheckpointLoaderSimple, 
    EmptyLatentImage, 
    KSampler, 
    VAEDecode, 
    CLIPTextEncode, 
    SaveImage,
    PreviewImage,
)

In [6]:
seed = random.randint(1, 2**32)
seed

3573699943

In [20]:
positive = 'beautiful scenery nature glass bottle landscape, , purple galaxy bottle,'
negative = 'text, watermark'
image_height = 1024 * 3
image_width = 1024 * 3
model_name = 'Realistic_Vision_V6.0_NV_B1.safetensors'
scheduler = 'karras'
sampler_name = 'euler'
seed = 2412009591
steps = 20
cfg = 8

In [21]:
with torch.inference_mode():
    
    # load model checkpoint
    model, clip, vae = CheckpointLoaderSimple(model_name)
    
    # clip encode
    positive_prompt = CLIPTextEncode(positive, clip)
    negative_prompt = CLIPTextEncode(negative, clip)
    
    latent = EmptyLatentImage(image_width, image_height, 1)
    latent = KSampler(
        model, 
        seed, 
        steps, 
        cfg, 
        sampler_name, 
        scheduler, 
        positive_prompt, 
        negative_prompt, 
        latent
    )
    
    image = VAEDecode(latent, vae)
    
    # PreviewImage(image)
    output_image = SaveImage(image, 'Notebook', extra_pnginfo={'myfield': {'positive': positive, 'negative': negative}})
    
    Image.open(f"../output/{output_image['ui']['images'][0]['filename']}")

  0%|          | 0/20 [00:00<?, ?it/s]